In [1]:
import json
import pandas as pd
import dspy
import os

json_file_path = '../results/pubmed_results.json'

with open(json_file_path, 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

In [3]:
from datasets import Dataset 
dataset = Dataset.from_dict(loaded_data)

In [7]:
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness,answer_relevancy,answer_similarity,context_precision,context_recall,context_entity_recall,summarization_score
import time 
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"]
from langchain_openai.chat_models import ChatOpenAI
from ragas.embeddings import HuggingfaceEmbeddings

gpt4 = ChatOpenAI(model="GPT-4o-mini-2024-07-18")


hf_embeddings = HuggingfaceEmbeddings(model_name="BAAI/bge-small-en")

#summarizartion score needs the summary field
#metrics =[answer_correctness,answer_relevancy,answer_similarity,faithfulness,context_precision,context_recall,context_entity_recall,noise_sensitivity_relevant,
#           noise_sensitivity_irrelevant,reference_free_rubrics_score, labelled_rubrics_score]
batch_size = 50
num = 1
all_results = []
for i in range(0, 200, batch_size):
    score = evaluate(dataset.select(range(i,min(i+batch_size,200))),metrics=[faithfulness],embeddings=hf_embeddings) 
    result_df = score.to_pandas()
    all_results.append(result_df)
    result_df.to_parquet(f'../results/metric_pubmed_{num}.parquet')
    time.sleep(1000)
    num = num + 1

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [21]:
# Concatenate all DataFrames in the list
df = pd.concat(all_results, ignore_index=True)

In [26]:
df1= pd.DataFrame.from_dict(loaded_data)

In [29]:
df = df.rename(columns={'user_input': 'question'})

merged_df = pd.merge(df, df1[['question', 'id']], on='question', how='left')

In [30]:
merged_df.head()

,question,retrieved_contexts,response,reference,faithfulness,id
0,Is ganglionated plexi ablation during Maze IV ...,[We investigated the role of surgical ablation...,The absence of ganglionated plexi ablation was...,No. GP ablation did not prove to be beneficial...,1.000000,25985014
1,Production of chemokines by perivascular adipo...,[Obesity is associated with an increased risk ...,"Yes, perivascular adipose tissue (pWAT) may pl...",Yes. Human pWAT has chemotactic properties thr...,1.000000,16195477
2,Is a 9-month treatment sufficient in tuberculo...,[Tuberculosis has increased in parallel with t...,"Yes, a 9-month treatment was found to be suffi...",Yes. Tuberculous enterocolitis can be managed ...,0.666667,12848629
3,A patient with myelomeningocele: is untetherin...,[Tethering of the spinal cord is thought to in...,The study found that patients with myelomening...,No. The study results suggested that spinal co...,0.800000,20594006
4,Do some U.S. states have higher/lower injury m...,[This article examines the hypothesis that the...,"Yes, some U.S. states have higher/lower injury...",Yes. Group 1 states are likely to exhibit abov...,0.750000,15995461


In [31]:
merged_df.to_json('../results/ragas_results.json', orient='records', force_ascii=False, indent=4)